# Web Scrapping Assignment-3

In [1]:
!pip install selenium
!pip install geopy

In [65]:
# Importing Libraries
import selenium
import pandas as pd
import time
from bs4 import BeautifulSoup


# Importing selenium webdriver 
from selenium import webdriver


# Importing required Exceptions which needs to handled
from selenium.common.exceptions import StaleElementReferenceException, NoSuchElementException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys

#Importing requests
import requests

# importing regex
import re

# 1. Write a python program which searches all the product under a particular product vertical from www.amazon.in. The product verticals to be searched will be taken as input from user. For e.g. If user input is ‘guitar’. Then search for guitars.

In [14]:
#loading the drivers and URL

driver=webdriver.Chrome('chromedriver.exe')
driver.get('http://amazon.in/')


# Creating the inpiut for the product name to be entered
product=input("Enter the product name: ")

#updating the input details in seracg=h bar
driver.find_element_by_id("twotabsearchtextbox").send_keys(product)
driver.find_element_by_id("nav-search-submit-button").click()

Enter the product name: laptop


# 2.In the above question, now scrape the following details of each product listed in first 3 pages of your search results and save it in a data frame and csv. In case if any product vertical has less than 3 pages in search results then scrape all the products available under that product vertical. Details to be scraped are: "Brand Name", "Name of the Product", "Rating", "No. of Ratings", "Price", "Return/Exchange", "Expected Delivery", "Availability", "Other Details" and “Product URL”. In case, if any of the details are missing for any of the product then replace it by “-“.

In [15]:
#to update the url of all products from 3 pages
urls=[]
for i in range(3):
    for j in driver.find_elements_by_xpath("//div[@class='a-section a-spacing-medium']//h2"):
        urls.append(j.find_element_by_xpath(".//a").get_attribute('href'))
    try:
        next_page=driver.find_element_by_xpath("//div[@class='a-text-center']/ul/li[@class='a-last']/a").get_attribute('href')
        driver.get(next_page)
        driver.refresh()   
    except:
        break

In [16]:

#creating the empty list
brand=[]
name=[]
rating=[]
num_of_rating=[]
price=[]
return_exc=[]
exp_del=[]
avail=[]
pro_url=[]

#updating all the prodcut urls
for i in urls:
    driver.get(i)
    pro_url.append(i)
    time.sleep(2)
    
#Scrapping the brand name
    try:
        brand.append(driver.find_element_by_id("productTitle").text.split(' ')[0])
        
    except NoSuchElementException:
        brand.append("-")
            
#Scrapping the prodcut name
    try:
        name.append(driver.find_element_by_id("productTitle").text.split(' ')[1:]) 
    except NoSuchElementException:
        name.append("-")
            
#Scrapping the product rating
    try:
        rating.append(driver.find_element_by_xpath('//*[@id="a-popover-content-8"]/div/div/div/div[1]/span').text)
    except NoSuchElementException:
        rating.append("-")

#Scrapping the number of rating
    try:
        num_of_rating.append(driver.find_element_by_xpath('//*[@id="acrCustomerReviewText"]').text)
    except NoSuchElementException:
        num_of_rating.append("-")
    
#Scrapping the price
    try: 
        try: price.append(driver.find_element_by_id("priceblock_saleprice").text) 
        except:
            try: price.append(driver.find_element_by_id("priceblock_dealprice").text)
            except: price.append(driver.find_element_by_id("priceblock_ourprice").text)
    except: price.append('-')
            
#scrapping for return/exchange
    try:
        return_exc.append(driver.find_element_by_xpath('//*[@id="RETURNS_POLICY"]/span/div[2]/a').text)
    except NoSuchElementException:
        return_exc.append("-")
            
#scrapping for expected delivery
    try:
        exp_del.append(driver.find_element_by_xpath('//*[@id="ddmDeliveryMessage"]/b').text)
    except NoSuchElementException:
        exp_del.append("-")

#scrapping for availability
    try:
        avail.append(driver.find_element_by_id("availability").text)
    except NoSuchElementException:
        avail.append("-")

In [17]:
#creating the DataFrame
Product={'Brand':brand,'Name':name,'Rating':rating,'Number of rating':num_of_rating,'Price':price,
        'Return/Exchange':return_exc,'Expected Delivery':exp_del,'Availability':avail,'URL':pro_url}
Product=pd.DataFrame(data=Product)
Product

,Brand,Name,Rating,Number of rating,Price,Return/Exchange,Expected Delivery,Availability,URL
0,Lenovo,"[IdeaPad, Slim, 3, 10th, Gen, Intel, Core, i3,...",-,180 ratings,"₹39,490.00",7 Days Replacement,"Tuesday, Aug 24",In stock.,https://www.amazon.in/gp/slredirect/picassoRed...
1,Lenovo,"[IdeaPad, Slim, 3, 2021, 11th, Gen, Intel, Cor...",-,93 ratings,"₹41,990.00",7 Days Replacement,"Tuesday, Aug 24",In stock.,https://www.amazon.in/gp/slredirect/picassoRed...
2,AVITA,"[Essential, NE14A2INC433-CR, 14"", (35.56cms)La...",-,"1,059 ratings","₹22,990.00",7 Days Replacement,"Friday, Aug 27",Only 1 left in stock.,https://www.amazon.in/AVITA-Essential-NE14A2IN...
3,ASUS,"[VivoBook, 15, (2020),, 39.6, cm, HD,, Dual, C...",-,423 ratings,"₹24,990.00",7 Days Replacement,Aug 25 - 27,"In stock on August 23, 2021.\nOrder it now.",https://www.amazon.in/ASUS-VivoBook-Integrated...
4,HP,"[15, AMD, Athlon, 15.6"", (39.62cms), HD, Lapto...",-,319 ratings,"₹29,630.00",7 Days Replacement,"Wednesday, Aug 25",In stock.,https://www.amazon.in/HP-Athlon-15-6-inch-Silv...
...,...,...,...,...,...,...,...,...,...
61,(Renewed),"[Dell, Latitude, 6520, Core, i5, 2nd, Gen, |, ...",-,-,"₹22,999.00",7 Days Replacement,Aug 22 - 28,In stock.,https://www.amazon.in/Renewed-Dell-Latitude-Nu...
62,HP,"[14, 11th, Gen, Intel, Core, i5, Processor, 14...",-,402 ratings,"₹58,990.00",7 Days Replacement,"Tuesday, Aug 24",In stock.,https://www.amazon.in/HP-Processor-14-inch-Win...
63,HP,"[14, (2021), Thin, &, Light, 11th, Gen, Intel,...",-,75 ratings,"₹46,990.00",7 Days Replacement,"Tuesday, Aug 24",In stock.,https://www.amazon.in/HP-Laptop-14-inch-Window...
64,Microsoft,"[Surface, Pro, 7, VDV-00015, 12.3"", (31.24, cm...",-,174 ratings,"₹87,000.00",7 Days Replacement,"Friday, Aug 27",Only 1 left in stock.,https://www.amazon.in/gp/slredirect/picassoRed...


# 3.Write a python program to access the search bar and search button on images.google.com and scrape 100 images each for keywords ‘fruits’, ‘cars’ and ‘Machine Learning’.

In [2]:
# We will create empty list in which we will update the required data
fruits=[]
cars=[]
machine=[]

In [3]:
#creating the list for the name of the items we want the images
Name=['fruits','cars','machine']

In [4]:
#loading the drivers and url
driver=webdriver.Chrome("chromedriver.exe")
driver.maximize_window()
wait = WebDriverWait(driver,15)
driver.get("https://www.google.co.in/imghp?hl=en&ogbl")

In [5]:
for i in Name:
    if i=='fruits':
        driver.find_element_by_xpath("//input[@class='gLFyf gsfi']").send_keys(i) #entering the item for search
        driver.find_element_by_xpath("//button[@class='Tg7LZd']").click()

In [6]:
#creating the instance to scroll down the page for images to load.
SCROLL_PAUSE_TIME = 2
CYCLES = 10

html = driver.find_element_by_tag_name('html')
html.send_keys(Keys.PAGE_DOWN)  
html.send_keys(Keys.PAGE_DOWN)  
time.sleep(SCROLL_PAUSE_TIME * 3)

for j in range(CYCLES):
    html.send_keys(Keys.END)
    time.sleep(SCROLL_PAUSE_TIME)

In [7]:
#scrapping for fruits            
fruit=driver.find_elements_by_xpath("//a[@class='VFACy kGQAp sMi44c lNHeqe WGvvNb']")
for k in fruit:
    fruits.append(k.get_attribute('href'))
html.send_keys(Keys.HOME)
driver.find_element_by_xpath("//input[@class='og3lId']").clear() #clearing the data in searc bar

In [8]:
#scrapping for cars
for i in Name:
    if i=='cars':
        driver.find_element_by_xpath("//input[@class='og3lId']").send_keys(i)
        driver.find_element_by_xpath("//button[@class='rCGXm']").click()

In [9]:
SCROLL_PAUSE_TIME = 2  
CYCLES = 10

html = driver.find_element_by_tag_name('html')
html.send_keys(Keys.PAGE_DOWN)  
html.send_keys(Keys.PAGE_DOWN)  
time.sleep(SCROLL_PAUSE_TIME * 3)

for j in range(CYCLES):
    html.send_keys(Keys.END)
    time.sleep(SCROLL_PAUSE_TIME)

In [10]:
car=driver.find_elements_by_xpath("//a[@class='VFACy kGQAp sMi44c lNHeqe WGvvNb']")
for k in car:
    cars.append(k.get_attribute('href'))
html.send_keys(Keys.HOME)
driver.find_element_by_xpath("//input[@class='og3lId']").clear()

In [11]:
#scrapping for machine

if i=='machine':
    driver.find_element_by_xpath("//input[@class='og3lId']").send_keys(i)
    driver.find_element_by_xpath("//button[@class='rCGXm']").click()
        
    SCROLL_PAUSE_TIME = 2
    CYCLES = 10

    html = driver.find_element_by_tag_name('html')
    html.send_keys(Keys.PAGE_DOWN)  
    html.send_keys(Keys.PAGE_DOWN)  
    time.sleep(SCROLL_PAUSE_TIME * 3)

    for j in range(CYCLES):
        html.send_keys(Keys.END)
        time.sleep(SCROLL_PAUSE_TIME)
mac=driver.find_elements_by_xpath("//a[@class='VFACy kGQAp sMi44c lNHeqe WGvvNb']")
for k in mac:
    machine.append(k.get_attribute('href'))

In [12]:
#creating the dataframe
Google={'Fruits':fruits,'Cars':cars,'Machine':machine}
Google=pd.DataFrame(data=Google)
Google[:100]

,Fruits,Cars,Machine
0,https://www.amazon.in/Fruits-All-One-Dreamland...,https://www.extremetech.com/extreme/303740-car...,https://en.wikipedia.org/wiki/Machine
1,https://www.goodhousekeeping.com/health/diet-n...,https://www.cardekho.com/newcars,https://www.indiamart.com/proddetail/wire-nail...
2,https://www.youtube.com/watch?v=PY2OxP_jMZ0,https://www.motor1.com/car-lists/coolest-cars/,https://www.ies.co.uk/news/10-iconic-machines-...
3,https://in.pinterest.com/pin/649081365039047796/,https://www.ft.com/content/8089bb56-30fe-11ea-...,https://www.macmillandictionaryblog.com/machine
4,https://www.healthyeating.org/nutrition-topics...,https://www.popularmechanics.com/cars/a3482425...,https://www.amazon.in/Special-Overlock-Machine...
...,...,...,...
95,https://www.nutritionadvance.com/healthy-foods...,https://www.indiatvnews.com/auto/top-10-cars-o...,https://www.youtube.com/watch?v=vrQrsCpCCS4
96,https://www.elsevier.com/books/fresh-cut-fruit...,https://cars.tatamotors.com/,https://www.kwt-auto.com/
97,https://theveganatlas.com/12-unusual-exotic-fr...,https://www.aitrends.com/ai-insider/top-end-sp...,https://www.iberdrola.com/innovation/machine-l...
98,https://www.liveworksheets.com/worksheets/en/E...,https://www.carwale.com/,https://www.industry-plaza.com/automatic-singl...


# 4.Write a python program to search for a smartphone(e.g.: Oneplus Nord, pixel 4A, etc.) on www.flipkart.com and scrape following details for all the search results displayed on 1st page. Details to be scraped: “Brand Name”, “Smartphone name”, “Colour”, “RAM”, “Storage(ROM)”, “Primary Camera”, “Secondary Camera”, “Display Size”, “Display Resolution”, “Processor”, “Processor Cores”, “Battery Capacity”, “Price”, “Product URL”. Incase if any of the details is missing then replace it by “- “. Save your results in a dataframe and CSV.

In [13]:

#loading the driver and the url
driver=webdriver.Chrome('chromedriver.exe')
time.sleep(2)
url=('https://www.flipkart.com/')
driver.get(url)
time.sleep(2)
driver.find_element_by_xpath("//button[@class='_2KpZ6l _2doB4z']").click()
time.sleep(2)

In [14]:
#creating the user input to search for mobiles
user_inp= input('Enter the product you want to search: ')
driver.find_element_by_xpath("//div[@class='_3OO5Xc']/input").send_keys(user_inp)
time.sleep(2)
driver.find_element_by_xpath("//button[@class='L0Z3Pu']").click()
time.sleep(2)

Enter the product you want to search: smartphone


In [15]:
#creating the empty list
Brand=[] 
Smartphone_name=[]
Colour=[]
RAM=[] 
Storage=[]
Primary_Camera=[]
Secondary_Camera=[]
Display_Size=[]
Display_Resolution=[]
Processor=[]
Processor_Cores=[]
Battery_Capacity=[]
Price=[]
Product_url=[]

In [16]:
#clicking on the product link
product_links=driver.find_elements_by_xpath("//a[@class='_1fQZEK']")

#scrapping the product link
for i in product_links:
    Product_url.append(i.get_attribute('href'))
for i in Product_url:
    driver.get(i)
    driver.find_element_by_xpath('//button[@class="_2KpZ6l _1FH0tX"]').click()
#scrapping the Brand name
    try:
        brand=driver.find_element_by_xpath('//span[@class="B_NuCI"]').text.split()
        Brand.append(brand[0])
    except:
        Brand.append('-')
#scrapping other details
    a= driver.find_elements_by_xpath("//li[@class='_21lJbe']")
    b=driver.find_elements_by_xpath("//td[@class='_1hKmbr col col-3-12']")
    for i in range(0,len(b)):
        if b[i].text=='Model Name':
            Smartphone_name.append(a[i].text)        
        if b[i].text=='Color':
            Colour.append(a[i].text)
        if b[i].text=='RAM':
            RAM.append(a[i].text)
        if b[i].text=='Internal Storage':
            Storage.append(a[i].text)
        if b[i].text=='Primary Camera':
            Primary_Camera.append(a[i].text)
        if b[i].text=='Secondary Camera':
            Secondary_Camera.append(a[i].text)                                  
        if b[i].text=='Display Size':
            Display_Size.append(a[i].text)
        if b[i].text=='Resolution':
            Display_Resolution.append(a[i].text)
        if b[i].text=='Processor Type':
            Processor.append(a[i].text)            
        if b[i].text=='Processor Core':
            Processor_Cores.append(a[i].text)
        if b[i].text=='Battery Capacity':
            Battery_Capacity.append(a[i].text)
#scrapping the prices
    try:
        price=driver.find_element_by_xpath('//div[@class="_30jeq3 _16Jk6d"]')
        Price.append(price.text)
    except:
        Price.append('-')

In [17]:
#creating the dataframe
Details={'Brand Name':Brand,'Smartphone Name':Smartphone_name,'Color':Colour,'RAM':RAM,'Storage(ROM)':Storage,
        'Primary Camera':Primary_Camera,'Secondary Camera':Secondary_Camera,'Display Size':Display_Size,
         'Display Resolution':Display_Resolution,'Processor':Processor,'Processor Cores':Processor_Cores,
        'Battery Capacity':Battery_Capacity,'Price':Price}

Details=pd.DataFrame(data=Details)
Details

,Brand Name,Smartphone Name,Color,RAM,Storage(ROM),Primary Camera,Secondary Camera,Display Size,Display Resolution,Processor,Processor Cores,Battery Capacity,Price
0,realme,RMX3063 / RMX3061,C20,32 GB,"2G GSM: 850/900/1800/1900 MHz, 3G WCDMA: 850/9...",Yes,Yes,"Head: 0.982W/kg, Body: 0.497W/kg",16.51 cm (6.5 inch),Android 10,MediaTek Helio G35,"MP4, 3GP, 3G2, 3GPP, 3GPP2, M4V, MKV","₹7,499"
1,realme,RMX3063 / RMX3061,C20,32 GB,"2G GSM: 850/900/1800/1900 MHz, 3G WCDMA: 850/9...",Yes,Yes,"Head: 0.982W/kg, Body: 0.497W/kg",16.51 cm (6.5 inch),Android 10,MediaTek Helio G35,"MP4, 3GP, 3G2, 3GPP, 3GPP2, M4V, MKV","₹7,499"
2,REDMI,9i,Nature Green,4 GB,64 GB,13MP Rear Camera,5MP Front Camera,16.59 cm (6.53 inch),1600 x 720 Pixels,MediaTek Helio G25,Octa Core,5000 mAh,"₹8,499"
3,POCO,C3,Arctic Blue,3 GB,32 GB,13MP + 2MP + 2MP,5MP Front Camera,16.59 cm (6.53 inch),1600 x 720 Pixels,Mediatek Helio G35,Octa Core,5000 mAh,"₹7,499"
4,POCO,M3,Cool Blue,4 GB,64 GB,48MP + 2MP + 2MP,8MP Front Camera,16.59 cm (6.53 inch),2340 x 1080 Pixels,Qualcomm Snapdragon 662,Octa Core,6000 mAh,"₹10,499"
5,POCO,M3,Cool Blue,6 GB,64 GB,48MP + 2MP + 2MP,8MP Front Camera,16.59 cm (6.53 inch),2340 x 1080 Pixels,Qualcomm Snapdragon 662,Octa Core,6000 mAh,"₹11,499"
6,POCO,M3,Power Black,4 GB,64 GB,48MP + 2MP + 2MP,8MP Front Camera,16.59 cm (6.53 inch),2340 x 1080 Pixels,Qualcomm Snapdragon 662,Octa Core,6000 mAh,"₹10,499"
7,POCO,C3,Matte Black,3 GB,32 GB,13MP + 2MP + 2MP,5MP Front Camera,16.59 cm (6.53 inch),1600 x 720 Pixels,Mediatek Helio G35,Octa Core,5000 mAh,"₹7,499"
8,Infinix,Smart 5,Aegean Blue,2 GB,32 GB,13MP + Low Light Sensor,8MP Front Camera,17.32 cm (6.82 inch),1640 x 720 Pixels,MediaTek Helio G25,Octa Core,6000 mAh,"₹6,999"
9,Infinix,Smart 5,Morandi Green,2 GB,32 GB,13MP + Low Light Sensor,8MP Front Camera,17.32 cm (6.82 inch),1640 x 720 Pixels,MediaTek Helio G25,Octa Core,6000 mAh,"₹6,999"


# 5.Write a program to scrap geospatial coordinates (latitude, longitude) of a city searched on google maps.

In [18]:
driver=webdriver.Chrome('chromedriver.exe')
time.sleep(2)
driver.get('https://www.google.com/maps/')

City = input('Enter City Name : ')
driver.find_element_by_xpath("//input[@class='tactile-searchbox-input']").send_keys(City)
driver.find_element_by_xpath("//button[@class='searchbox-searchbutton']").click()

latitude=driver.current_url.split('/@')[1].split(',')[0]
longtitude=driver.current_url.split('/@')[1].split(',')[1].split(',')[0]
print('Latitude:',latitude,'Longitude:',longtitude)

Enter City Name : jalpaiguri
Latitude: 24.372814 Longitude: 87.8504985


# 6.Write a program to scrap details of all the funding deals for second quarter (i.e. July 20 – September 20) from trak.in.

In [19]:
#loading the driver and the url
driver=webdriver.Chrome('chromedriver.exe')
driver.get('https://trak.in/')
time.sleep(2)

driver.get(driver.find_element_by_xpath('//li[@id="menu-item-51510"]/a').get_attribute('href'))

In [20]:
#creating the empty list 
date=[]
name=[]
industry=[]
sub=[]
city=[]
investor=[]
inv_type=[]
amount=[]

#scrapping for date
for i in driver.find_elements_by_xpath('//td[@class="column-2"]'):
    date.append(i.text)

#scrapping for name
for i in driver.find_elements_by_xpath('//td[@class="column-3"]'):
    name.append(i.text)

#scrapping for industry    
for i in driver.find_elements_by_xpath('//td[@class="column-4"]'):
    industry.append(i.text)

#scrapping for sub industry
for i in driver.find_elements_by_xpath('//td[@class="column-5"]'):
    sub.append(i.text)

#scrapping for city    
for i in driver.find_elements_by_xpath('//td[@class="column-6"]'):
    city.append(i.text)

#scrapping for investor
for i in driver.find_elements_by_xpath('//td[@class="column-7"]'):
    investor.append(i.text)

#scrapping for investor type    
for i in driver.find_elements_by_xpath('//td[@class="column-8"]'):
    inv_type.append(i.text)

#scrapping for amount
for i in driver.find_elements_by_xpath('//td[@class="column-9"]'):
    amount.append(i.text)

In [21]:
#checking for length of each columns
len(date),len(name),len(industry),len(sub),len(city),len(investor),len(inv_type),len(amount)

(392, 392, 392, 392, 392, 392, 392, 392)

In [22]:
#creating the dataframe for Jul-Sep
Funding={'Date':date,'Name':name,'Industry/Vertica':industry,'Sub-vertical':sub,'City':city,'Investor':investor,
        'Investment Type':inv_type,'Amount':amount}
Funding=pd.DataFrame(data=Funding)
Funding[55:85]

,Date,Name,Industry/Vertica,Sub-vertical,City,Investor,Investment Type,Amount
55,08/09/2020,Byju’s,EduTech,Online Tutoring,Bangalore,"Silver Lake, Tiger Global, General Atlantic an...",Private Equity,"500,000,000"
56,12/09/2020,mCaffeine,Personal Care,Skincare & Haircare,Mumbai,"Amicus Capital Private Equity I LLP, Amicus Ca...",Series B,"3,000,000"
57,09/09/2020,Qshala,EduTech,Online Curiosity Platform for Kids,Bangalore,Rainmatter Capital,Angel,"370,000"
58,02/09/2020,Winzo,Online Gaming,Online Gaming,New Delhi,"Kalaari Capital Partners, IndigoEdge Managemen...",Series B,"15,500,000"
59,09/09/2020,Hippo Video,Video Customer Experience(CX) Platform,Video Customer Experience(CX) Platform,"Newark, Delaware, United States of Amercia","Alpha Wave Incubation, Exfinity Venture Partne...",Series A,"4,500,000"
60,07/09/2020,Melorra,E-commerce,Online Jewelry Store,Bangalore,"Shadow Holdings, Lightbox.",Debt Financing,"upto 8,900,000"
61,07/09/2020,1mg,E-commerce,Online Pharmacy,Gurgaon,"Gaja Capital, Tata Capital, Partners Group",In Progress,"100,000,000"
62,31/08/2020,mfine,HealthTech,On-Demand Healthcare Services,Bangalore,Caretech Pte Inc,Series B,"5,400,000"
63,31/08/2020,Apna,Human Resources,Recruitment Platform,Bangalore,Lightspeed India and Sequoia Capital India,Series A,"8,000,000"
64,03/09/2020,Railofy,Transportation,WL & RAC protection platform,Mumbai,Chiratae Ventures,Seed,"950,000"


# 7.Write a program to scrap all the available details of best gaming laptops from digit.in.

In [33]:
driver= webdriver.Chrome("chromedriver.exe")
url= 'https://www.digit.in/'
driver.get(url)
time.sleep(2)

In [34]:
top_10=driver.find_element_by_xpath("/html/body/div[1]/div/div[4]/ul/li[4]")
top_10.click()

In [35]:
laptop=driver.find_element_by_xpath("/html/body/div[3]/div/div/div[3]/div[2]/ul/li[26]")
laptop.click()

In [36]:
product_name=[]
p_n=driver.find_elements_by_xpath("//div[@class='right-container']/div/a/h3")
for i in p_n:
    product_name.append(i.text)
product_name

['ALIENWARE AREA 51M R2',
 'ALIENWARE M15 R3',
 'ASUS ROG STRIX SCAR 15',
 'ASUS ROG ZEPHYRUS G14',
 'LENOVO LEGION 5I',
 'ASUS ROG ZEPHYRUS DUO 15',
 'ACER ASPIRE 7 GAMING']

In [37]:
details=[]
o_s=driver.find_elements_by_xpath('//div[@class="product-detail"]')
for i in o_s:
    details.append(i.text.replace('\n','-'))
details

['OS-WINDOWS 10 HOME-DISPLAY-17.3" (1920 X 1080)-PROCESSOR-10TH GENERATION INTEL® CORE™ I7-10700 | 2.90 GHZ-MEMORY-1 TB SSD/16 GBGB DDR4-FULL SPECS OTHER ALIENWARE LAPTOPS',
 'OS-WINDOWS 10 HOME-DISPLAY-15.6" (3840 X 2160)-PROCESSOR-10TH GENERATION INTEL® CORE™ I9-10980HK | NA-MEMORY-1 TB SSD/16 GBGB DDR4-FULL SPECS OTHER ALIENWARE LAPTOPS',
 'OS-WINDOWS 10 HOME-DISPLAY-15.6" (1920 X 1080)-PROCESSOR-AMD RYZEN™ 9 5900HX | 3.3 GHZ-MEMORY-1 TB SSD/16 GBGB DDR4-READ REVIEW FULL SPECS OTHER ASUS LAPTOPS',
 'OS-WINDOWS 10 HOME-DISPLAY-14" (1920 X 1080)-PROCESSOR-AMD 3RD GENERATION RYZEN 9 | 3.3 GHZ-MEMORY-1 TB SSD/16 GBGB DDR4-READ REVIEW FULL SPECS OTHER ASUS LAPTOPS',
 'OS-WINDOWS 10 PRO-DISPLAY-15.6" (1920 X 1080)-PROCESSOR-10TH GENERATION INTEL® CORE™ I5-10300H | 2.50 GHZ-MEMORY-1 TB SSD/16 GBGB DDR4-FULL SPECS OTHER LENOVO LAPTOPS',
 'OS-WINDOWS 10-DISPLAY-15.6" (3840 X 1100)-PROCESSOR-INTEL CORE I7 10TH GEN 10875H | NA-MEMORY-512 GB SSD/4 GBGB DDR4-FULL SPECS OTHER ASUS LAPTOPS',
 'OS-

In [38]:
digit_lap=pd.DataFrame()
digit_lap['product_name']=product_name
digit_lap['details']=details
digit_lap

,product_name,details
0,ALIENWARE AREA 51M R2,"OS-WINDOWS 10 HOME-DISPLAY-17.3"" (1920 X 1080)..."
1,ALIENWARE M15 R3,"OS-WINDOWS 10 HOME-DISPLAY-15.6"" (3840 X 2160)..."
2,ASUS ROG STRIX SCAR 15,"OS-WINDOWS 10 HOME-DISPLAY-15.6"" (1920 X 1080)..."
3,ASUS ROG ZEPHYRUS G14,"OS-WINDOWS 10 HOME-DISPLAY-14"" (1920 X 1080)-P..."
4,LENOVO LEGION 5I,"OS-WINDOWS 10 PRO-DISPLAY-15.6"" (1920 X 1080)-..."
5,ASUS ROG ZEPHYRUS DUO 15,"OS-WINDOWS 10-DISPLAY-15.6"" (3840 X 1100)-PROC..."
6,ACER ASPIRE 7 GAMING,"OS-WINDOWS 10 HOME-DISPLAY-15.6"" (1920 X 1080)..."


# 8.Write a python program to scrape the details for all billionaires from www.forbes.com. Details to be scrapped: “Rank”, “Name”, “Net worth”, “Age”, “Citizenship”, “Source”, “Industry”.

In [55]:
#loading the drivers and url
driver=webdriver.Chrome("chromedriver.exe") 
time.sleep(3)

driver.get('https://www.forbes.com')

driver.find_element_by_xpath("//button[@class='icon--hamburger']").click() # clicking on the hamburget button

#getting the billionaire page
billionaire=driver.find_element_by_xpath("//li[@class='header__channel header__color--centennial-silver header__hoverable']/a")
driver.get(billionaire.get_attribute('href'))
time.sleep(2)

driver.find_element_by_xpath("//h2[@class='chansec-special-feature__title no-label--special-feature']").click()
time.sleep(2)

#creating the empty list
Rank=[]
Name=[]
Net_worth=[]
Age=[]
Citizenship=[]
Source=[]
Industry=[]

#there are 15 pages hence we will scrap through range function
start=0
end=15
for page in range(start,end):
    #scrapping for rank
    rank=driver.find_elements_by_xpath("//div[@class='rank']")
    for i in rank:
        Rank.append(i.text)

    #scrapping for name
    name=driver.find_elements_by_xpath("//div[@class='personName']")
    for i in name:
        Name.append(i.text)

    #scrapping for Networth
    networth=driver.find_elements_by_xpath("//div[@class='netWorth']/div")
    for i in networth:
        Net_worth.append(i.text)
    
    #scrapping for age
    age=driver.find_elements_by_xpath("//div[@class='age']/div")
    for i in age:
        Age.append(i.text)
    
    #scrapping for citizenship
    citi=driver.find_elements_by_xpath("//div[@class='countryOfCitizenship']")
    for i in citi:
        Citizenship.append(i.text)

    #scrapping for source
    source=driver.find_elements_by_xpath("//div[@class='source-column']/div[1]")
    for i in source:
        Source.append(i.text)
    
    #scrapping for industry
    industry=driver.find_elements_by_xpath("//div[@class='category']/div")
    for i in industry:
        Industry.append(i.text)
    
    #to go on the next page
    try:
        nxt=driver.find_element_by_xpath("//button[@class='pagination-btn pagination-btn--next ']").click()
    except:
        NoSuchElementException

In [56]:
#creating the dataframe
Forbes_Billionarie={'Rank':Rank,'Name':Name,'Net Worth':Net_worth,'Age':Age,
                    'Citizenship':Citizenship,'Source':Source,'Industry':Industry}
Forbes_Billionarie=pd.DataFrame(data=Forbes_Billionarie)
Forbes_Billionarie

,Rank,Name,Net Worth,Age,Citizenship,Source,Industry


# 9.Write a program to extract at least 500 Comments, Comment upvote and time when comment was posted from any YouTube Video.

In [49]:
#creating the empty list
Comments=[]
Upvote=[]
Updated=[]

#loading the drivers and url
driver=webdriver.Chrome("chromedriver.exe")
wait = WebDriverWait(driver,15)
driver.get("https://www.youtube.com/watch?v=ukzFI9rgwfU")

#creating the program to scroll down the comments and let it load
SCROLL_PAUSE_TIME = 2
CYCLES = 100

html = driver.find_element_by_tag_name('html')
html.send_keys(Keys.PAGE_DOWN)  
html.send_keys(Keys.PAGE_DOWN)  
time.sleep(SCROLL_PAUSE_TIME * 3)

for i in range(CYCLES):
    html.send_keys(Keys.END)
    time.sleep(SCROLL_PAUSE_TIME)

#scrapping for comments
comment = driver.find_elements_by_xpath('//*[@id="content-text"]')
for i in comment:
    Comments.append(i.text)

#scrapping for upvote
upvote = driver.find_elements_by_xpath('//*[@id="vote-count-middle"]')
for i in upvote:
    Upvote.append(i.text)

#scrapping for updated on
updated=driver.find_elements_by_xpath('//*[@id="header-author"]/yt-formatted-string/a')
for i in updated:
    Updated.append(i.text)

#creating the dataframe
Youtube={'Comments':Comments,'Comment Upvote':Upvote, 'Updated on':Updated}
Youtube=pd.DataFrame(data=Youtube)
Youtube

,Comments,Comment Upvote,Updated on
0,**Summary**:\n- machine learning is the genera...,2.7K,2 years ago
1,Scenario 1: Facebook recognizes your friend in...,1K,1 year ago
2,I'm impressed by the way you taught. Teacher s...,215,11 months ago
3,Literally learnt more from you than 4 years in...,101,3 months ago
4,"I am from a health care background, but I coul...",30,1 year ago
...,...,...,...
591,Alexa,,1 year ago
592,such an awful voice. thats why i disliked ihat...,,6 months ago
593,1. Unsupervised\n2. Supervised \n3. Unsupervised,,2 years ago
594,you should have use a better voice over withou...,1,2 years ago


# 10.Write a python program to scrape a data for all available Hostels from https://www.hostelworld.com/ in “London” location. You have to scrape hostel name, distance from city centre, ratings, total reviews, overall reviews, privates from price, dorms from price, facilities and property description.

In [62]:
driver= webdriver.Chrome("chromedriver.exe")
url='https://www.hostelworld.com/s?q=London,%20England&country=England&city=London&type=city&id=3&from=2021-07-08&to=2021-07-11&guests=2&page=1'
driver.get(url)

In [67]:
hn=driver.find_elements_by_xpath('//h2[@class="title title-6"]')
name=[]
for i in hn:
    name.append(i.text)

d=driver.find_elements_by_xpath('//span[@class="description"]')
distance=[]
for i in d:
    distance.append(i.text)

r=driver.find_elements_by_xpath('//div[@class="score orange big"]')
rating=[]
for i in r:
    rating.append(i.text)

to_r=driver.find_elements_by_xpath('//div[@class="reviews"]')
total_reviews=[]
for i in to_r:
    total_reviews.append(i.text)

facilities=[]
f = driver.find_elements_by_xpath('//div[@class="facilities-label facilities"]')
for i in f:
    facilities.append(i.text)

prices=[]
p = driver.find_elements_by_xpath('//div[@class="price-col"]')
for i in p:
    prices.append(i.text.replace('\n','-'))


hostel_url=[]
urls = driver.find_elements_by_xpath('//h2[@class="title title-6"]//a')
for i in urls:
    hostel_url.append(url.get_attribute("href"))


AttributeError: 'str' object has no attribute 'get_attribute'